# Traffic Sign Recognition Model
## Environment Initialisation

In [2]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [3]:
# Libraries
import os
import sys
import tensorflow as tf
import numpy as np
import math
IntialiseEnv()
import import_ipynb
import pickle
from sklearn.model_selection import train_test_split

from Data_Preparation.Data_Preparation import LoadTrainDataSetWithAugmentation
from Data_Preparation.Data_Preparation import LoadTrainDataSet
from Data_Preparation.Data_Preparation import LoadTestDataSet

importing Jupyter notebook from C:\Users\Owner\Desktop\Cairo University\Masters\Semester 1\Deep Learning\Project\traffic_sign_env\Traffic_Sign_Recognition_Detection\Source_Code\Data_Preparation\Data_Preparation.ipynb
importing Jupyter notebook from C:\Users\Owner\Desktop\Cairo University\Masters\Semester 1\Deep Learning\Project\traffic_sign_env\Traffic_Sign_Recognition_Detection\Source_Code\Pre_Processing\SegmentationNew.ipynb
importing Jupyter notebook from C:\Users\Owner\Desktop\Cairo University\Masters\Semester 1\Deep Learning\Project\traffic_sign_env\Traffic_Sign_Recognition_Detection\Source_Code\Pre_Processing\Segmentation.ipynb


## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [4]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
/device:GPU:0
Num GPUs Available:  1


## Reading DataSet
Pre-processing and Loading Train and Test DataSet respectively

In [5]:
def ReadTrainDataSet():
    print("Reading Train Pre-processed DataSet")
    processedTrainDataSet = 'Processed_DataSet/TrainDataSet.pkl'
    isProcessedTrainDataSetExits= os.path.exists(processedTrainDataSet)
    train_image_array, train_image_labels = None, None
    
    if isProcessedTrainDataSetExits:
        print("Loading Processed Train DataSet from Processed_DataSet/TrainDataSet.pkl")
        file = open(processedTrainDataSet, 'rb')
        train_image_array, train_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Train DataSet.")
    else:
        print("Processed_DataSet/TrainDataSet.pkl file does not exist")
        print("Loading Train DataSet ... This may take a while.")
        train_image_array, train_image_labels =  LoadTrainDataSetWithAugmentation()
        file = open(processedTrainDataSet, 'wb')
        pickle.dump((train_image_array, train_image_labels), file, protocol=4)
        file.close()
        print("Saving pre-processed train DataSets in Processed_DataSet/TrainDataSet.pkl")
    return train_image_array, train_image_labels    

In [8]:
def ReadTestDataSet():
    processedTestDataSet = 'Processed_DataSet/TestDataSet.pkl'
    isProcessedTestDataSetExits= os.path.exists(processedTestDataSet)
    test_image_array, test_image_labels = None, None
    
    if isProcessedTestDataSetExits:
        print("Loading Processed Test DataSet from Processed_DataSet/TestDataSet.pkl")
        file = open(processedTestDataSet, 'rb')
        test_image_array, test_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Test DataSet.")
    else:
        print("Processed_DataSet/TestDataSet.pkl file does not exist")
        print("Loading Test DataSet ... This may take a while.")
        test_image_array, test_image_labels = LoadTestDataSet()
        file = open(processedTestDataSet, 'wb')
        pickle.dump((test_image_array, test_image_labels), file)
        file.close()
        print("Saving pre-processed test DataSets in Processed_DataSet/TestDataSet.pkl")
    return test_image_array, test_image_labels

In [9]:
train_image_array, train_image_labels= ReadTrainDataSet()

Reading Train Pre-processed DataSet
Loading Processed Train DataSet from Processed_DataSet/TrainDataSet.pkl
Done Loading Train DataSet.


In [10]:
test_image_array, test_image_labels= ReadTestDataSet()

Loading Processed Test DataSet from Processed_DataSet/TestDataSet.pkl
Done Loading Test DataSet.


## Train-Split 
Split Train images dataset into two splits: training and validation respectively 90-10

In [ ]:
image_train, image_valid,label_train, label_valid = train_test_split(train_image_array, train_image_labels,stratify=train_image_labels,test_size=0.1,random_state=43)

## Hyper-parameters Tuning

In [ ]:
# Hyper-parameters tuning
kernel_1 = (5,5)
kernel_2 = (3,3)
pooling = (2,2)
dropout = 0.2
num_classes = 43

## Model Architecture

In [ ]:
def model1():
    model = None
    tf.initializers.Orthogonal(gain=1.0, seed=None)
    
    layers = [
        tf.keras.layers.Conv2D(32,kernel_2, input_shape=(60,60,1), strides = 1, padding='valid',activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32,kernel_2, activation = tf.nn.relu),
        tf.keras.layers.MaxPool2D(pooling),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(128,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128,kernel_2,  activation = tf.nn.relu),
        tf.keras.layers.MaxPool2D(pooling),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(256,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(256,kernel_2,  activation = tf.nn.relu),
        tf.keras.layers.MaxPool2D(pooling),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.BatchNormalization(),

       
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation = tf.nn.relu),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
    model = tf.keras.Sequential(layers)
    return model

## Model Optimizer 

In [ ]:
model = model1()

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

## Model Training

In [ ]:
history = model.fit(image_train, np.array(label_train) , batch_size=32, epochs=90, validation_data=(image_valid, np.array(label_valid)))

## Visualization

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Model Summary 
Number of parameters used in the model architecture

In [ ]:
print(model.summary())

## Model Evaluation

In [13]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=128)

12630/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.03988623887290168, 0.99247825]

## Saving Trained .h5 Model

In [ ]:
def saveCNNModel(model_name):
    model.save(model_name)

In [ ]:
saveCNNModel()

In [ ]:
model._set_inputs(image_train,np.array(label_train))

## Loading Saved Trained .h5 Model

In [11]:
def load_model():
    Model = tf.keras.models.load_model('Trained_Models/winning_model_9925.h5')
    return Model

In [12]:
del model
model= load_model()

## Computation Avg. Processing Time

In [ ]:
import time
t1=time.time()
model.predict(test_image_array)
t2=time.time()
print("Average Processing time: ", ((t2-t1)/12630)*1000, " ms")